In [ ]:
%pwd

In [ ]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from VGG_image import * #2
from VGG_model import * #1
import torch
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import cv2
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist
from scipy import spatial
import operator
cmap = plt.cm.jet

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='0' # set the GPU id

In [ ]:
subf = 'iter1'
tgtf = 'iter2'
iter_k = 1 #set for select the new added samples

#K = 20 # num for uncertainty
#k = 10 # num for representativeness

root = 'YOUR/DIRECTORY/OF/ACTIVE/SELECTION/FOLDER'
dMap_folder = './VGG_dmaps_' + subf 
res_file = './VGG_results_' + subf + '.txt'

transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [ ]:
#testing set
adi_test_path = os.path.join(root,'test','images')
test_path_sets = [adi_test_path]

#validation set
sub_path = 'val'
adi_val_path = os.path.join(root, sub_path, 'images')
val_path_sets = [adi_val_path]

#unlabeled seting
unlabeled_path = 'train_unlabeled_' + subf
adi_unlabeled = os.path.join(root, unlabeled_path, 'images')
unlabeled_sets = [adi_unlabeled]

test_paths = []
for path in test_path_sets:
    for img_path in glob.glob(os.path.join(path, '*.png')):
        test_paths.append(img_path)

val_paths = []
for path in val_path_sets:
    for img_path in glob.glob(os.path.join(path, '*.png')):
        val_paths.append(img_path)

unlabeled_train_paths = []
for path in unlabeled_sets:
    for img_path in glob.glob(os.path.join(path, '*.png')):
        unlabeled_train_paths.append(img_path)        
        

model1 = Generator()
model2 = Generator()
model3 = Generator()

model1 = model1.cuda()
model2 = model2.cuda()
model3 = model3.cuda()

saved_ckp1 = './VGG_sa_n10_m1_model_best.pth.tar' #choose the best
saved_ckp2 = './VGG_sa_n10_m2_model_best.pth.tar' #choose the best
saved_ckp3 = './VGG_sa_n10_m3_model_best.pth.tar' #choose the best

checkpoint1 = torch.load(saved_ckp1)
checkpoint2 = torch.load(saved_ckp2)
checkpoint3 = torch.load(saved_ckp3)

model1.load_state_dict(checkpoint1['g_state_dict'])
model2.load_state_dict(checkpoint2['g_state_dict'])
model3.load_state_dict(checkpoint3['g_state_dict'])

loader = transforms.Compose([transforms.ToTensor()])  
unloader = transforms.ToPILImage()

def imshow(tensor, title=None):
    image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
    image = image.squeeze(0)  # remove the fake batch dimension
    image = unloader(image)

if not os.path.isdir(dMap_folder):
    os.mkdir(dMap_folder)

In [ ]:
len(unlabeled_train_paths), len(test_paths) # of testing examples

In [ ]:
mae1 = 0.0
mse1 = 0.0
mae2 = 0.0
mse2 = 0.0

for i in range(len(test_paths)):
    file_path = test_paths[i]
    file_name = file_path.split('/')[-1].split('.')[0]
    image_name1 = os.path.join(dMap_folder, file_name + '_m1_dmap.png')
    image_name2 = os.path.join(dMap_folder, file_name + '_m2_dmap.png')
    image_name3 = os.path.join(dMap_folder, file_name + '_m3_dmap.png')

    img = transform(Image.open(test_paths[i]).convert('RGB')).cuda()
    gt_file = h5py.File(test_paths[i].replace('.png','.h5').replace('images','ground_truth'),'r')
    groundtruth = np.asarray(gt_file['density'])
    
    output1, _ = model1(img.unsqueeze(0))
    output2, _ = model2(img.unsqueeze(0))
    output3, _ = model3(img.unsqueeze(0))
   
    output_re1 = output1.view(256,-1)
    output_re2 = output2.view(256,-1)
    output_re3 = output3.view(256,-1)
    
    image1 = output1.cpu().clone()
    image1 = image1.squeeze(0)
    img1 = transforms.ToPILImage()(image1) # use to calculate the variance

    image2 = output2.cpu().clone()
    image2 = image2.squeeze(0)
    img2 = transforms.ToPILImage()(image2)
    
    image3 = output3.cpu().clone()
    image3 = image3.squeeze(0)
    img3 = transforms.ToPILImage()(image3) 
    
    k1 = output_re1.cpu().clone().detach().numpy()
    k2 = output_re2.cpu().clone().detach().numpy()
    k3 = output_re3.cpu().clone().detach().numpy()

    plt.imsave(image_name1, k1, cmap=cmap) 
    plt.imsave(image_name2, k2, cmap=cmap) 
    plt.imsave(image_name3, k3, cmap=cmap)
    
    out_num1 = output1.data.cpu().clone().detach().numpy().sum()
    out_num2 = output2.data.cpu().clone().detach().numpy().sum() 
    out_num3 = output3.data.cpu().clone().detach().numpy().sum() 

    gt_num = np.around(np.sum(groundtruth))
   
    # mean average value
    mean_diff1 = (out_num1 + out_num2 + out_num3) / 3.0
    mae_val1 = abs(mean_diff1 - gt_num)
    mse_val1 = abs(mean_diff1 - gt_num)*abs(mean_diff1 - gt_num)
    
    # avg.
    mae1 += mae_val1
    mse1 += mse_val1

    
    with open(res_file, 'a+') as f:
        out_name = file_name
        f.write("{}: image: {}, MAE: {m_val1:.3f}, p1: {out1:.3f}, p2: {out2:.3f}, p3: {out3:.3f}, p_avg:{out_p:.3f}, GT:{gt:.3f}".format(i+1, file_name, m_val1=mae_val1, out1=out_num1, out2=out_num2, out3=out_num3, out_p=mean_diff1, gt=gt_num))
        f.write('\n')
    print ("{}: image: {}, MAE: {m_val1:.3f}, p1: {out1:.3f}, p2: {out2:.3f}, p3: {out3:.3f}, p_avg:{out_p:.3f}, GT:{gt:.3f}".format(i+1, file_name, m_val1=mae_val1, out1=out_num1, out2=out_num2, out3=out_num3, out_p=mean_diff1, gt=gt_num))
print ('TestingSet Average MAE_<avg> error: ', mae1/len(test_paths)) 
print ('TestingSet Average MSE_<avg> error: ', np.sqrt(mse1/len(test_paths)))
print ('********************************************')
print ('TestingSet Average MAE_<best> error: ', mae2/len(test_paths)) 
print ('TestingSet Average MSE_<best> error: ', np.sqrt(mse2/len(test_paths)))

with open(res_file, 'a+') as f:
    f.write('TestingSet Average MAE_<avg> error: {} \n'.format(mae1/len(test_paths)))
    f.write('TestingSet Average MSE_<avg> error: {} \n'.format(np.sqrt(mse1/len(test_paths))))
    f.write('TestingSet Average MAE_<best> error: {} \n'.format(mae2/len(test_paths)))
    f.write('TestingSet Average MSE_<best> error: {} \n'.format(np.sqrt(mse2/len(test_paths))))

In [ ]:
# get uncertainty score of each image, method-1
scores = {}
clusters = {}
keys = []

for i in range(len(unlabeled_train_paths)):
    file_path = unlabeled_train_paths[i]
    file_name = file_path.split('/')[-1].split('.')[0]
    img = transform(Image.open(unlabeled_train_paths[i]).convert('RGB')).cuda()
    
    output1, cluster1 = model1(img.unsqueeze(0))
    output2, cluster2 = model2(img.unsqueeze(0))
    output3, cluster3 = model3(img.unsqueeze(0))

    est1 = output1.cpu().clone().detach().numpy().sum()
    est2 = output2.cpu().clone().detach().numpy().sum()
    est3 = output3.cpu().clone().detach().numpy().sum()
    
    cluster1 = np.argmax(cluster1.cpu().clone().detach().numpy())
    cluster2 = np.argmax(cluster2.cpu().clone().detach().numpy())
    cluster3 = np.argmax(cluster3.cpu().clone().detach().numpy())
    #print("three clusters are:", cluster1, cluster2, cluster3)
    ests = [est1, est2, est3]
    scores[file_name] = np.std(ests, axis=0)
    keys.append(file_name)
    
    # for clusters:
    if cluster1 == cluster2 == cluster3:
        clus = cluster1
    elif cluster1 == cluster2 and cluster1 != cluster3:
        clus = cluster1
    elif cluster1 == cluster3 and cluster1 != cluster2:
        clus = cluster1
    elif cluster2 == cluster3 and cluster2 != cluster1:
        clus = cluster2
    else:
        # re-assign the cluster to the new patch
        clus = 10
    clusters[file_name] = clus

In [ ]:
# get uncertainty score of each image, method-2
scores = {}
clusters = {}
keys = []


# load saved checkpoints
model1_1 = Generator().cuda()
model1_2 = Generator().cuda()
model1_3 = Generator().cuda()

model2_1 = Generator().cuda()
model2_2 = Generator().cuda()
model2_3 = Generator().cuda()

model3_1 = Generator().cuda()
model3_2 = Generator().cuda()
model3_3 = Generator().cuda()

saved_ckp1_1 = './VGG_sa_n10_m1_20_checkpoint.pth.tar' 
saved_ckp1_2 = './VGG_sa_n10_m1_40_checkpoint.pth.tar' 
saved_ckp1_3 = './VGG_sa_n10_m1_60_checkpoint.pth.tar' 

saved_ckp2_1 = './VGG_sa_n10_m2_20_checkpoint.pth.tar'
saved_ckp2_2 = './VGG_sa_n10_m2_40_checkpoint.pth.tar'
saved_ckp2_3 = './VGG_sa_n10_m2_60_checkpoint.pth.tar'

saved_ckp3_1 = './VGG_sa_n10_m3_20_checkpoint.pth.tar'
saved_ckp3_2 = './VGG_sa_n10_m3_40_checkpoint.pth.tar'
saved_ckp3_3 = './VGG_sa_n10_m3_60_checkpoint.pth.tar'

checkpoint1_1 = torch.load(saved_ckp1_1)
checkpoint1_2 = torch.load(saved_ckp1_2)
checkpoint1_3 = torch.load(saved_ckp1_3)

checkpoint2_1 = torch.load(saved_ckp2_1)
checkpoint2_2 = torch.load(saved_ckp2_2)
checkpoint2_3 = torch.load(saved_ckp2_3)

checkpoint3_1 = torch.load(saved_ckp3_1)
checkpoint3_2 = torch.load(saved_ckp3_2)
checkpoint3_3 = torch.load(saved_ckp3_3)

model1_1.load_state_dict(checkpoint1_1['g_state_dict'])
model1_2.load_state_dict(checkpoint1_2['g_state_dict'])
model1_3.load_state_dict(checkpoint1_3['g_state_dict'])

model2_1.load_state_dict(checkpoint2_1['g_state_dict'])
model2_2.load_state_dict(checkpoint2_2['g_state_dict'])
model2_3.load_state_dict(checkpoint2_3['g_state_dict'])

model3_1.load_state_dict(checkpoint3_1['g_state_dict'])
model3_2.load_state_dict(checkpoint3_2['g_state_dict'])
model3_3.load_state_dict(checkpoint3_3['g_state_dict'])


for i in range(len(unlabeled_train_paths)):
    file_path = unlabeled_train_paths[i]
    file_name = file_path.split('/')[-1].split('.')[0]
    img = transform(Image.open(unlabeled_train_paths[i]).convert('RGB')).cuda()
    
    output1_1, cluster1_1 = model1_1(img.unsqueeze(0))
    output1_2, cluster1_2 = model1_2(img.unsqueeze(0))
    output1_3, cluster1_3 = model1_3(img.unsqueeze(0))
    
    output2_1, cluster2_1 = model2_1(img.unsqueeze(0))
    output2_2, cluster2_2 = model2_2(img.unsqueeze(0))
    output2_3, cluster2_3 = model2_3(img.unsqueeze(0))
    
    output3_1, cluster3_1 = model3_1(img.unsqueeze(0))
    output3_2, cluster3_2 = model3_2(img.unsqueeze(0))
    output3_3, cluster3_3 = model3_3(img.unsqueeze(0))

    est1_1 = output1_1.cpu().clone().detach().numpy().sum()
    est1_2 = output1_2.cpu().clone().detach().numpy().sum()
    est1_3 = output1_3.cpu().clone().detach().numpy().sum()
    
    est2_1 = output2_1.cpu().clone().detach().numpy().sum()
    est2_2 = output2_2.cpu().clone().detach().numpy().sum()
    est2_3 = output2_3.cpu().clone().detach().numpy().sum()
    
    est3_1 = output3_1.cpu().clone().detach().numpy().sum()
    est3_2 = output3_2.cpu().clone().detach().numpy().sum()
    est3_3 = output3_3.cpu().clone().detach().numpy().sum()
    
    cluster1 = np.argmax(cluster1_3.cpu().clone().detach().numpy())
    cluster2 = np.argmax(cluster2_3.cpu().clone().detach().numpy())
    cluster3 = np.argmax(cluster3_3.cpu().clone().detach().numpy())
    #print("three clusters are:", cluster1, cluster2, cluster3)
    
    ests_1 = [est1_1, est2_1, est3_1]
    ests_2 = [est1_2, est2_2, est3_2]
    ests_3 = [est1_3, est2_3, est3_3]
    
    ests_e1 = np.std(ests_1, axis=0)
    ests_e2 = np.std(ests_2, axis=0)
    ests_e3 = np.std(ests_3, axis=0)
    
    ests = [ests_e1, ests_e2, ests_e3]
    scores[file_name] = np.std(ests, axis=0)
    keys.append(file_name)
    
    # for clusters:
    if cluster1 == cluster2 == cluster3:
        clus = cluster1
    elif cluster1 == cluster2 and cluster1 != cluster3:
        clus = cluster1
    elif cluster1 == cluster3 and cluster1 != cluster2:
        clus = cluster1
    elif cluster2 == cluster3 and cluster2 != cluster1:
        clus = cluster2
    else:
        # re-assign the cluster to the new patch
        clus = 10
    clusters[file_name] = clus

In [ ]:
import os, csv
cluster_root = "YOUR/CLUSTER/FILE/FOLDER"
cluster_file = "clustering_train_labels.csv"
results = {}
with open(os.path.join(cluster_root, cluster_file), newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        results[row[0]] = int(row[1])
# get exisitng files and clusters
old_files = list(results.keys())
old_clusters = list(results.values())
new_clusters = list(set(old_clusters))

In [ ]:
len(old_clusters)

In [ ]:
# for all existing file and clusters
clus_total = [[] for i in range(len(new_clusters)+1)] # 10 clusters
for idx1 in range(10):
    for idx2 in range(len(old_clusters)):
        if old_clusters[idx2] == idx1:
            clus_total[idx1].append(old_files[idx2])
existing_list = clus_total[:-1]

In [ ]:
clus_total

In [ ]:
len(existing_list)

In [ ]:
# for new validation set & labeled train data
new_files = list(clusters.keys())
new_clusters = list(clusters.values())
for idx in range(len(clusters.keys())):
    idx_clus = new_clusters[idx]
    clus_total[idx_clus].append(new_files[idx])

In [ ]:
# for each cluster, compute the mean value vector (cetroid) for first 10 clusters
vgg_root = "/YOUR/DATASET/DIRECTORY"

centroids = [] # length is 10
for i in range(10):
    patch_fea = [] 
    for idx in range(len(clus_total[i])):
        fea_temp = []
        file_name = clus_total[i][idx] + ".png"
        patch_img = os.path.join(vgg_root, file_name)
        img = transform(Image.open(patch_img).convert('RGB')).cuda()

        model1.backend1.register_forward_hook(get_activation('backend1'))
        output1 = model1(img.unsqueeze(0))
        fea1 = activation['backend1']
        fea1 = fea1.squeeze(0).cpu().clone().detach().numpy()
        encoding1 = np.mean(fea1, axis=0).flatten()

        model2.backend1.register_forward_hook(get_activation('backend1'))
        output2 = model2(img.unsqueeze(0))
        fea2 = activation['backend1']
        fea2 = fea2.squeeze(0).cpu().clone().detach().numpy()
        encoding2 = np.mean(fea2, axis=0).flatten()

        model3.backend1.register_forward_hook(get_activation('backend1'))
        output3 = model3(img.unsqueeze(0))
        fea3 = activation['backend1']
        fea3 = fea3.squeeze(0).cpu().clone().detach().numpy()
        encoding3 = np.mean(fea3, axis=0).flatten()

        fea_temp = [encoding1, encoding2, encoding3]
        patch_fea.append(np.mean(fea_temp, axis=0)) # get average feature of one patch
    centroids.append(np.mean(patch_fea, axis=0))

In [ ]:
len(clus_total)

In [ ]:
# only copy the new insert unlabeled data
clus_total_copy = clus_total[:-1]
for idx in range(len(clus_total_copy)):
    clus_total_copy[idx] = clus_total[idx][iter_k:]
    print(len(clus_total_copy[idx]), len(clus_total[idx]))

In [ ]:
clus_total[10]

In [ ]:
# undecidable samples
from scipy.spatial import distance
for idx in range(len(clus_total[10])):
   
    file_name = clus_total[10][idx] + ".png"
    patch_img = os.path.join(vgg_root, file_name)
    img = transform(Image.open(patch_img).convert('RGB')).cuda()
    
    model1.backend1.register_forward_hook(get_activation('backend1'))
    output1 = model1(img.unsqueeze(0))
    fea1 = activation['backend1']
    fea1 = fea1.squeeze(0).cpu().clone().detach().numpy()
    encoding1 = np.mean(fea1, axis=0).flatten()

    model2.backend1.register_forward_hook(get_activation('backend1'))
    output2 = model2(img.unsqueeze(0))
    fea2 = activation['backend1']
    fea2 = fea2.squeeze(0).cpu().clone().detach().numpy()
    encoding2 = np.mean(fea2, axis=0).flatten()

    model3.backend1.register_forward_hook(get_activation('backend1'))
    output3 = model3(img.unsqueeze(0))
    fea3 = activation['backend1']
    fea3 = fea3.squeeze(0).cpu().clone().detach().numpy()
    encoding3 = np.mean(fea3, axis=0).flatten()
    
    fea_temp = [encoding1, encoding2, encoding3]
    patch_fea = np.mean(fea_temp, axis=0) # get average feature of one patch

    dist_list = []
    for idx1 in range(len(centroids)):
        dist = np.linalg.norm(patch_fea - centroids[idx1])
        #dist = distance.cosine(patch_fea, centroids[idx1])
        dist_list.append(dist)
    soft_cluster = np.argmin(dist_list) # 0-9
    #print(dist_list)
    #print(soft_cluster)
    clus_total_copy[soft_cluster].append(clus_total[10][idx]) # append the patch name 

In [ ]:
# check each cluster number of samples
sum = 0
min_val = 200000
for idx in range(10):
    if len(clus_total_copy[idx]) < min_val:
        min_val = len(clus_total_copy[idx])
    sum += len(clus_total_copy[idx])
    print("the class {} has {} samples".format(idx+1, len(clus_total_copy[idx]))) 
sum, min_val

In [ ]:
existing_features = [[] for i in range(len(existing_list))]

for idx in range(len(existing_list)):
    len1 = len(existing_list[idx])
    for idx1 in range(len1):
        file_name = existing_list[idx][idx1] + ".png"
        patch_img = os.path.join(vgg_root, file_name)
        img = transform(Image.open(patch_img).convert('RGB')).cuda()

        model1.backend1.register_forward_hook(get_activation('backend1'))
        output1 = model1(img.unsqueeze(0))
        fea1 = activation['backend1']
        fea1 = fea1.squeeze(0).cpu().clone().detach().numpy()
        encoding1 = np.mean(fea1, axis=0).flatten()

        model2.backend1.register_forward_hook(get_activation('backend1'))
        output2 = model2(img.unsqueeze(0))
        fea2 = activation['backend1']
        fea2 = fea2.squeeze(0).cpu().clone().detach().numpy()
        encoding2 = np.mean(fea2, axis=0).flatten()

        model3.backend1.register_forward_hook(get_activation('backend1'))
        output3 = model3(img.unsqueeze(0))
        fea3 = activation['backend1']
        fea3 = fea3.squeeze(0).cpu().clone().detach().numpy()
        encoding3 = np.mean(fea3, axis=0).flatten()

        fea_temp = [encoding1, encoding2, encoding3]
        patch_fea = np.mean(fea_temp, axis=0) # get single image patch feature
        existing_features[idx].append(patch_fea)        

In [ ]:
# for each new clustered samples, compute the similarity
sim_list = [{} for i in range(len(existing_features))]
for idx in range(len(existing_features)):
    for idx1 in range(len(clus_total_copy[idx])):
        file_name = clus_total_copy[idx][idx1] + ".png"
        patch_img = os.path.join(vgg_root, file_name)
        img = transform(Image.open(patch_img).convert('RGB')).cuda()

        model1.backend1.register_forward_hook(get_activation('backend1'))
        output1 = model1(img.unsqueeze(0))
        fea1 = activation['backend1']
        fea1 = fea1.squeeze(0).cpu().clone().detach().numpy()
        encoding1 = np.mean(fea1, axis=0).flatten()

        model2.backend1.register_forward_hook(get_activation('backend1'))
        output2 = model2(img.unsqueeze(0))
        fea2 = activation['backend1']
        fea2 = fea2.squeeze(0).cpu().clone().detach().numpy()
        encoding2 = np.mean(fea2, axis=0).flatten()

        model3.backend1.register_forward_hook(get_activation('backend1'))
        output3 = model3(img.unsqueeze(0))
        fea3 = activation['backend1']
        fea3 = fea3.squeeze(0).cpu().clone().detach().numpy()
        encoding3 = np.mean(fea3, axis=0).flatten()

        fea_temp = [encoding1, encoding2, encoding3]
        patch_fea = np.mean(fea_temp, axis=0) # get average feature of one patch

        dist = 0.0
        for ix in range(len(existing_features[idx])):
            dist_1 = distance.cosine(patch_fea, existing_features[idx][ix]) # new feature vs. existing features
            dist += dist_1
        file1 = clus_total_copy[idx][idx1]
        sim_list[idx][file1] = dist

In [ ]:
sorted_dist = sorted(sim_list[0].items(), key=lambda x: -x[1])
sorted_dist

In [ ]:
#method1: considering the rarity of exisiting samples
new_select_file = []
new_label_list = []
for idx in range(len(sim_list)):
    sorted_dist = sorted(sim_list[idx].items(), key=lambda x: -x[1])
    new_list = sorted_dist[:min_val]
    cluster_score = {}
    for idx1 in range(len(new_list)):
        file_name = new_list[idx1][0]
        cluster_score[file_name] = scores[file_name]
    sorted_scores_list = sorted(cluster_score.items(), key=lambda x: -x[1])
    #print(sorted_scores_list)
    new_select_file.append(sorted_scores_list[0][0]) #file_name
    new_label_list.append(idx)
    print("Select new patch {} for cluster {}".format(sorted_scores_list[0], idx+1))  

In [ ]:
#method2: direct select the first one
new_select_file = []
new_label_list = []
for idx in range(len(clus_total_copy)):
    cluster_score = {}
    for idx1 in range(len(clus_total_copy[idx])):
        file_name = clus_total_copy[idx][idx1]
        cluster_score[file_name] = scores[file_name]
    sorted_scores_list = sorted(cluster_score.items(), key=lambda x: -x[1])
    new_select_file.append(sorted_scores_list[0][0]) #file_name
    new_label_list.append(idx)
    print("Select new patch {} for cluster {}".format(sorted_scores_list[0], idx+1))

In [ ]:
new_select_file, new_label_list

In [ ]:
# iteration-1: copy files
from shutil import copytree

src_path_train = '../YOUR/ACTIVE/SELECTION/FOLDER_SOURCE/TRAIN/' + 'train_labeled_' + subf + '/' 
tgt_path_train = '../YOUR/ACTIVE/SELECTION/FOLDER_TARGET/TRAIN/' + 'train_labeled_' + tgtf + '/' 
copytree(src_path_train, tgt_path_train, dirs_exist_ok=True)
src_path_val = '../YOUR/ACTIVE/SELECTION/FOLDER_SOURCE/VAL/' + 'train_unlabeled_' + subf + '/' 
tgt_path_val = '../YOUR/ACTIVE/SELECTION/FOLDER_TARGET/VAL/' + 'train_unlabeled_' + tgtf + '/'
copytree(src_path_val, tgt_path_val, dirs_exist_ok=True)

In [ ]:
len(new_select_file)

In [ ]:
# move files from val to train
from shutil import move
for file in new_select_file:
    src_mv_file = file + ".png"
    src_path1 = os.path.join(tgt_path_val + "images", src_mv_file) # from val folder
    tgt_path1 = os.path.join(tgt_path_train + "images", src_mv_file) # to train folder
    move(src_path1, tgt_path1)
    tar_f1 = file + ".h5"
    tar_f2 = file + ".png"
    #tar_f3 = file + "dots.txt"
    #tar_f4 = file + "_dots.h5"
    #tar_f5 = file + "cell.png"
    move(os.path.join(tgt_path_val + "ground_truth", tar_f1), os.path.join(tgt_path_train + "ground_truth", tar_f1))
    move(os.path.join(tgt_path_val + "ground_truth", tar_f2), os.path.join(tgt_path_train + "ground_truth", tar_f2))
    #move(os.path.join(tgt_path_val + "ground_truth", tar_f3), os.path.join(tgt_path_train + "ground_truth", tar_f3))
    #move(os.path.join(tgt_path_val + "ground_truth", tar_f4), os.path.join(tgt_path_train + "ground_truth", tar_f4))
    #move(os.path.join(tgt_path_val + "ground_truth", tar_f5), os.path.join(tgt_path_train + "ground_truth", tar_f5))

In [ ]:
# write the selected file into CSV file
import csv
import os
with open(os.path.join(cluster_root,cluster_file), "a") as csv_file:   
    writer = csv.writer(csv_file, delimiter=',')
    level_counter = 0
    max_levels = 10
    while level_counter < max_levels:
        writer.writerow((new_select_file[level_counter], new_label_list[level_counter])) 
        level_counter = level_counter +1